# 作業 : (Kaggle)鐵達尼生存預測
[https://www.kaggle.com/c/titanic](https://www.kaggle.com/c/titanic)

In [30]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'Data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 作業1  
參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [31]:
#類別值 (object) 型欄位, 存於 object_features 中
#數值 (number) 型欄位, 存於 number_features 中
object_features = []
number_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
    else:
        number_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')
print(f'{len(number_features)} Numeric Features : {number_features}\n')

# 觀察欄位相異值數量
df.select_dtypes(include=["object"]).apply(pd.Series.nunique)

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64

**標籤編碼+邏輯斯迴歸**

In [32]:
# 只留類別型欄位
df_ob = df[object_features]
df_ob = df_ob.fillna('None')
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df_ob.columns:
    df_temp[c] = LabelEncoder().fit_transform(df_ob[c])
df_ob = df_temp
df_ob.head()


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [33]:
# 只留數值型欄位
df_nu = df[number_features]
df_nu = df_nu.fillna(df_nu.mean()) #先用平均值替代
df_nu.head()
df = pd.concat([df_ob.reset_index(drop=True),df_nu.reset_index(drop=True)], axis=1)
train_num = train_Y.shape[0]
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,155,1,720,185,3,3,22.0,1,0,7.2500
1,286,0,816,106,0,1,38.0,1,0,71.2833
2,523,0,914,185,3,3,26.0,0,0,7.9250
3,422,0,65,70,3,1,35.0,1,0,53.1000
4,22,1,649,185,3,3,35.0,0,0,8.0500


In [38]:
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7890443143400547


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,155,1,720,185,3,3,22.0,1,0,7.2500
1,286,0,816,106,0,1,38.0,1,0,71.2833
2,523,0,914,185,3,3,26.0,0,0,7.9250
3,422,0,65,70,3,1,35.0,1,0,53.1000
4,22,1,649,185,3,3,35.0,0,0,8.0500


**特徵雜湊+邏輯斯迴歸**

In [39]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7946497377284171


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash
0,155,1,720,185,3,3,22.0,1,0,7.2500,5
1,286,0,816,106,0,1,38.0,1,0,71.2833,6
2,523,0,914,185,3,3,26.0,0,0,7.9250,5
3,422,0,65,70,3,1,35.0,1,0,53.1000,0
4,22,1,649,185,3,3,35.0,0,0,8.0500,5


**計數編碼+特徵雜湊+邏輯斯迴歸**

In [42]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Cabin,Cabin_Count
185,185,1014
80,80,6
184,184,5
47,47,5
60,60,4
180,180,4
117,117,4
79,79,4
181,181,4
183,183,4


In [43]:
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,3,22.0,1,0,7.2500,5,1014
1,286,0,816,106,0,1,38.0,1,0,71.2833,6,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,5,1014
3,422,0,65,70,3,1,35.0,1,0,53.1000,0,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,5,1014


In [45]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df['Cabint_Count'] = df['Cabin_Count']
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7980271559518503


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash,Cabin_Count,Cabint_Count
0,155,1,720,185,3,3,22.0,1,0,7.2500,5,1014,1014
1,286,0,816,106,0,1,38.0,1,0,71.2833,6,2,2
2,523,0,914,185,3,3,26.0,0,0,7.9250,5,1014,1014
3,422,0,65,70,3,1,35.0,1,0,53.1000,0,2,2
4,22,1,649,185,3,3,35.0,0,0,8.0500,5,1014,1014


# 作業2   
承上題，三者比較效果何者最好?

標籤編碼+邏輯斯迴歸0.7890443143400547  
特徵雜湊+邏輯斯迴歸0.7946497377284171  
計數編碼+特徵雜湊+邏輯斯迴歸0.7980271559518503  
→以目標均值編碼展現效果最好

# 範例 : (Kaggle)鐵達尼生存預測
以下用鐵達尼生存預測資料, 觀察計數編碼與特徵雜湊的效果

In [22]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'Data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [24]:
# 觀察欄位相異值數量
df.select_dtypes(include=["object"]).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [25]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7788748937419742


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [26]:
# 加上 'Ticket' 欄位的計數編碼
count_df = df.groupby(['Ticket'])['Name'].agg({'Ticket_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on=['Ticket'], how='left')
count_df.sort_values(by=['Ticket_Count'], ascending=False).head(10)

D:\Anaconda\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Ticket,Ticket_Count
778,CA. 2343,11
104,1601,8
775,CA 2144,8
335,3101295,7
454,347077,7
459,347082,7
847,S.O.C. 14879,7
824,PC 17608,7
123,19950,6
49,113781,6


In [27]:
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1


In [28]:
# 'Ticket'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Count'] = df['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7811221556805532


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,2
4,22,1,649,185,3,1


In [30]:
# 'Ticket'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)  #特徵雜湊  0.0
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7766150776636674


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash
0,155,1,720,185,3,9
1,286,0,816,106,0,3
2,523,0,914,185,3,8
3,422,0,65,70,3,3
4,22,1,649,185,3,0


In [31]:
# 'Ticket'計數編碼 + 'Ticket'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
df_temp['Ticket_Count'] = df['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7777450211664924


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash,Ticket_Count
0,155,1,720,185,3,9,1
1,286,0,816,106,0,3,2
2,523,0,914,185,3,8,1
3,422,0,65,70,3,3,2
4,22,1,649,185,3,0,1


# 課後補充:  
[CSDN : Feature hashing(特徵哈希)](https://blog.csdn.net/laolu1573/article/details/79410187)  
[簡書 : 基於sklearn的文本特徵抽取 (計數編碼用於自然語言 : count vectorizer)](https://www.jianshu.com/p/063840752151)